# Importing libraries

In [1]:
import pandas as pd
import numpy as np

from category_encoders import TargetEncoder

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, fbeta_score, accuracy_score, roc_auc_score
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, GridSearchCV
from sklearn.compose import ColumnTransformer

from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE

import tensorflow
tensorflow.autograph.set_verbosity(0)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import Recall, AUC, Precision
import tensorflow_addons as tfa

/Users/jonw/miniforge3/envs/ml_env/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:54: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.2.0 and strictly below 2.4.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.4.0-rc0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


# Reading file and tidying

In [2]:
df_train = pd.read_csv("../dataset/train.csv")
df_test = pd.read_csv("../dataset/test.csv")

In [3]:
y_train = df_train["risk_flag"]
x_train = df_train.drop("risk_flag", axis=1)

y_test = df_test["risk_flag"]
x_test = df_test.drop("risk_flag", axis=1)

In [4]:
def target_encoding(df_x, df_y):
    x = df_x.copy()
    
    # Target Encoding — categorical columns with high cardinality: profession, city, state
    profession_target_enc = TargetEncoder()
    x["profession_encoded"] = profession_target_enc.fit_transform(x["profession"], df_y)
    
    city_target_enc = TargetEncoder()
    x["city_encoded"] = city_target_enc.fit_transform(x["city"], df_y)
    
    state_target_enc = TargetEncoder()
    x["state_encoded"] = state_target_enc.fit_transform(x["state"], df_y)
    
    x.drop("profession", axis=1, inplace=True)
    x.drop("city", axis=1, inplace=True)
    x.drop("state", axis=1, inplace=True)
    return x

x_train = target_encoding(x_train, y_train)
x_test = target_encoding(x_test, y_test)

/Users/jonw/miniforge3/envs/ml_env/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [5]:
scaler = MinMaxScaler()
x_train[['income','age','experience']] = scaler.fit_transform(x_train[['income','age','experience']])
x_test[['income','age','experience']] = scaler.fit_transform(x_test[['income','age','experience']])

x_train.shape

oversampler = SMOTE(random_state=2021)
x_train, y_train = oversampler.fit_resample(x_train, y_train)

input_shape = (9,)

# Changes shape to 3D
y_train = to_categorical(y_train, 2)
y_test = to_categorical(y_test, 2)    

# Create the model
model = Sequential()
model.add(Dense(120, input_shape=input_shape, activation='relu'))
model.add(Dense(120, activation='relu'))
model.add(Dense(120, activation='relu'))
model.add(Dense(2, activation='softmax'))

# Configure the model and start training
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=[Recall(), Precision(), tfa.metrics.FBetaScore(num_classes=2, beta=2.0), 'accuracy', AUC()])
model_fit = model.fit(x_train, y_train, epochs=5, batch_size=64, verbose=0, validation_split=0.2)
test_results = model.evaluate(x_test, y_test, verbose=1)

print()
print(f"-----------------------TEST SCORES-----------------------")
print(f"Loss: {test_results[0]}")
print(f"Recall: {test_results[1]}")
print(f"Precision: {test_results[2]}")
print(f"F2-Score: {test_results[3][0]}")
print(f"Accuracy score: {test_results[4]}")
print(f"AUC Score: {test_results[5]}")

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


2021-11-06 16:02:09.306422: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-11-06 16:02:09.306548: W tensorflow/core/platform/profile_utils/cpu_utils.cc:126] Failed to get CPU frequency: 0 Hz


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1575/1575 [==============================] - 1s 535us/step - loss: 0.5153 - recall: 0.7531 - precision: 0.7531 - fbeta_score: 0.6031 - accuracy: 0.7531 - auc: 0.8273

-----------------------TEST SCORES-----------------------
Loss: 0.5153264403343201
Recall: 0.7531150579452515
Precision: 0.7531150579452515
F2-Score: 0.8149861097335815
Accuracy score: 0.7531150579452515
AUC Score: 0.8273217082023621


In [7]:
test_results

[0.5153264403343201,
 0.7531150579452515,
 0.7531150579452515,
 array([0.8149861 , 0.39130434], dtype=float32),
 0.7531150579452515,
 0.8273217082023621]

In [6]:
# recall = make_scorer(recall_score, average="macro")
# f1 = make_scorer(f1_score , average='macro')

# def create_model(optimizer='adam', activation='relu'):
#     model = Sequential()
#     model.add(Dense(45, input_shape=(46,), activation=activation))
#     model.add(Dense(25, activation='relu'))
#     model.add(Dense(12, activation='relu'))
#     model.add(Dense(6, activation='relu'))
#     model.add(Dense(2, activation='sigmoid'))
    
#     model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy', Recall()])
#     model.compile(loss='binary_crossentropy', 
#                   optimizer='adam', 
#                   metrics=[Recall(), tfa.metrics.FBetaScore(num_classes=2, beta=2.0), AUC()])
#     return model

# # fix random seed for reproducibility
# seed = 0
# np.random.seed(seed)

# # create model
# model = KerasClassifier(build_fn=create_model, verbose=0)

# # define the grid search parameters
# epochs = [30, 50, 100]
# batch_size = [50, 100, 200, 250, 300, 350, 400]
# optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
# activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']

# param_grid = dict(epochs=epochs, optimizer=optimizer, batch_size=batch_size, activation=activation)

# grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3, verbose=1)

# grid_result = grid.fit(X_smote_train_scaled, y_smote_train_copy)

# # summarize results
# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# means = grid_result.cv_results_['mean_test_score']
# stds = grid_result.cv_results_['std_test_score']
# params = grid_result.cv_results_['params']
# for mean, stdev, param in zip(means, stds, params):
#     print("%f (%f) with: %r" % (mean, stdev, param))

NameError: name 'make_scorer' is not defined